In [ ]:
## Time Tests

@timeit
async def test_sequential_anomaly_insert(dataset_id, anomaly_records):
       #test adding an anomaly to a dataset
       for item in anomaly_records:
              async with httpx.AsyncClient(base_url=BASE_URL) as client:
                     anomaly_response = await client.post("/anomaly/create/"+dataset_id, json=item)

@timeit
async def test_gather_individual_anomaly_insert(dataset_id, anomaly_records):
       async with httpx.AsyncClient(base_url=BASE_URL) as client:
              anomalies = await asyncio.gather(*[client.post("/anomaly/create/"+dataset_id, json=anomaly) for anomaly in anomaly_records])
       return anomalies

@timeit
async def test_batch_insert_anomalies(dataset_id, anomaly_records):
       async with httpx.AsyncClient(base_url=BASE_URL) as client:
              anomaly_response = await client.post("/anomaly/create_many/"+dataset_id, json=anomaly_records)
       return anomaly_response

In [ ]:
#test adding collection
@timeit
async def add_collection_test(collection_name):
       collection_object=build_collection_object(collection_name)
       async with httpx.AsyncClient(base_url=BASE_URL) as client:
              response = await client.post("/collection/create", json=collection_object)
       return response.json()
       
@timeit
async def add_datafeed_test(collection_id):
       #test adding a dataset with the id of a created collection
       dataset_name="realKnownCause/nyc_taxi.csv"
       datafeed_object,anomaly_records=build_datafeed_object(dataset_name)
       async with httpx.AsyncClient(base_url=BASE_URL) as client:
              response = await client.post("/datafeed/create/"+collection_id, json={"payload":datafeed_object})
       return response.json(), anomaly_records

@timeit
async def add_prediction(dataset_response,detector_id):
       dataset_object={"name":dataset_response['name'],"id":dataset_response['id']}
       prediction=build_prediction_object(dataset_object,detector_id)

       #make scored dataset object
       async with httpx.AsyncClient(base_url=BASE_URL) as client: 
              response = await client.post("/prediction/create", json=prediction)

       return response

@timeit
async def add_anomalies(anomaly_entries,prediction_id,detector_id):
       for anomaly in anomaly_entries:
              anomaly["detector_id"]=detector_id
              anomaly["prediction_id"]=prediction_id
              print(anomaly)
       async with httpx.AsyncClient(base_url=BASE_URL) as client: 
              anomalies = await client.post("/anomaly/create_many", json=anomaly_entries)
       return anomalies

#response = await add_collection_test("test_nab")
print(response)
#collection_id=response['id']
#dataset_response, anomaly_records = await add_dataset_test(collection_id)

#prediction_response = await add_prediction(dataset_response,detector_id)

#anomalies= await add_anomalies(anomaly_records,prediction_response.json()['id'],detector_id)